In [1]:
import yaml

In [2]:
f = open('../config.yml')
preload = yaml.safe_load(f)
print(preload)

{'connection': {'omop': {'server': 'edw.shands.ufl.edu', 'database': 'DWS_OMOP'}, 'mtd': {'server': 'edw.shands.ufl.edu', 'database': 'DWS_METADATA'}}, 'preload': {'condition': 'preload_condition.sql', 'procedure': 'preload_procedure.sql', 'drug': {'order': 'preload_drug_order.sql', 'admin': 'preload_drug_admin.sql'}}}


In [3]:
list(preload['preload']['drug'].values())

['preload_drug_order.sql', 'preload_drug_admin.sql']

In [18]:
%run -i ../test_preload.py

FileNotFoundError: [Errno 2] No such file or directory: 'conf.yml'

In [4]:
import os
os.getcwd()

'z:\\OMOP\\omop_etl\\notebooks'

In [6]:
from pydma.databases import OneFLDb

In [8]:
import inspect
inspect.getfile(OneFLDb)

'Z:\\DMATools\\pydma\\pydma\\databases.py'